In [1]:


import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [6]:
import gc,inspect,json,re
import xml.etree.ElementTree as ET
from functools import partial
from typing import get_type_hints

import torch
import transformers

from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel,Field,validator




In [7]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
).eval()

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [17]:
def delete_model(*args):
  for var in args:
    if var in globals():
      del globals()[var]
  gc.collect()
  torch.cuda.empty_cache()


In [12]:
class BookRecommendation(BaseModel):
  """Provide book recommendation based on interest"""
  interest: str = Field(description="User's interest")
  recommended_book :str= Field(description="Recommended book")

  @validator("interest")
  def interests_must_not_be_empty(cls, v):
    if len(v) == 0:
      raise ValueError("interest must not be empty")
    return v


class Joke(BaseModel):
  """Provide a joke"""
  setup :str = Field(description = "Question to set up a joke")
  answer :str  = Field(description = "Answer to the joke")

  @validator("setup")
  def setup_must_not_be_empty(cls, v):
    if len(v) == 0:
      raise ValueError("setup must not be empty")
    return v

class SongRecommendation(BaseModel):
  """Provide song recommendation based on interest"""
  interest: str = Field(description="User's interest")
  recommended_song:str  = Field(description="Recommended song")

  @validator("interest")
  def interests_must_not_be_empty(cls, v):
    if len(v) == 0:
      raise ValueError("interest must not be empty")
    return v

In [13]:
convert_pydantic_to_openai_function(SongRecommendation)

<ipython-input-13-385687dcb70f>:1: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  convert_pydantic_to_openai_function(SongRecommendation)


{'name': 'SongRecommendation',
 'description': 'Provide song recommendation based on interest',
 'parameters': {'type': 'object',
  'properties': {'interest': {'description': "User's interest",
    'type': 'string'},
   'recommended_song': {'description': 'Recommended song', 'type': 'string'}},
  'required': ['interest', 'recommended_song']}}

In [14]:
convert_pydantic_to_openai_function(Joke)

{'name': 'Joke',
 'description': 'Provide a joke',
 'parameters': {'type': 'object',
  'properties': {'setup': {'description': 'Question to set up a joke',
    'type': 'string'},
   'answer': {'description': 'Answer to the joke', 'type': 'string'}},
  'required': ['setup', 'answer']}}

In [29]:

def extract_function_calls(completion):
    completion = completion.strip()
    pattern = r"((.*?))"
    match = re.search(pattern, completion, re.DOTALL)
    if not match:
        return None

    multiplefn = match.group(1)
    root = ET.fromstring(multiplefn)
    functions = root.findall("functioncall")
    return [json.loads(fn.text) for fn in functions]

In [37]:
import xml.etree.ElementTree as ET
import json

def extract_function_calls(text):
    """
    Extracts function calls from the given text.

    Args:
        text: The text to extract function calls from.

    Returns:
        A list of function calls.
    """

    if not text or not text.strip().startswith("<"):
        # Check if text starts with '{' as it might be a JSON
        if text.strip().startswith("{"):
            try:
                # Attempt to parse the text as JSON
                data = json.loads(text)
                # Assuming the function call is in the format: {"name": "function_name", "arguments": {...}}
                return [data] if isinstance(data, dict) and "name" in data else []
            except json.JSONDecodeError:
                # If JSON parsing fails, return an empty list
                return []
        else:
            # If not JSON and not XML, return an empty list
            return []

    try:
        root = ET.fromstring(text)
        # Extract function calls (replace with your original logic)
        # Assuming you want to extract elements with tag "function" and their "name" attribute
        function_calls = [
            {"name": element.get("name"), "arguments": element.get("arguments")}
            for element in root.findall(".//function")  # Find all "function" elements
        ]
        return function_calls
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return []  # Return an empty list to avoid further issues

In [38]:
def generate_hermes(prompt, model, tokenizer, generation_config_overrides={}):
    fn = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""
    prompt = f"""<|im_start|>system
You are a helpful assistant with access to the following functions:

{convert_pydantic_to_openai_function(Joke)}

{convert_pydantic_to_openai_function(BookRecommendation)}

{convert_pydantic_to_openai_function(SongRecommendation)}

To use these functions respond with:

     {fn}
     {fn}
    ...


Edge cases you must handle:
- If there are no functions that match the user request, you will respond politely that you cannot help.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant"""

    generation_config = model.generation_config
    generation_config.update(
        **{
            **{
                "use_cache": True,
                "do_sample": True,
                "temperature": 0.2,
                "top_p": 1.0,
                "top_k": 0,
                "max_new_tokens": 512,
                "eos_token_id": tokenizer.eos_token_id,
                "pad_token_id": tokenizer.eos_token_id,
            },
            **generation_config_overrides,
        }
    )

    model = model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    n_tokens = inputs.input_ids.numel()

    with torch.inference_mode():
        generated_tokens = model.generate(**inputs, generation_config=generation_config)

    return tokenizer.decode(
        generated_tokens.squeeze()[n_tokens:], skip_special_tokens=False
    )


In [39]:
generation_fun = partial(generate_hermes,model=model,tokenizer=tokenizer)

In [32]:
prompts = [
    "Tell me a joke",
    "Song for inspiration",
    "Recommend me a book on action"
]

In [22]:
for prompt in prompts:
  print(generation_fun(prompt))

{"name":"Joke","arguments":{"setup":"Why don't scientists trust atoms?","answer":"Because they make up everything."}}<|im_end|>

  {"name":"SongRecommendation","arguments":{"interest":"inspiration","recommended_song":"Eye of the Tiger by Survivor"}}<|im_end|>

  {"name":"BookRecommendation","arguments"{"interest":"action"}}<|im_end|>


In [40]:
generation_func = partial(generate_hermes, model=model, tokenizer=tokenizer)

prompts = [
    "Tell me a joke",
    "Song for inspiration.",
    "Recommend me a book on Crime Thriller."
]

for prompt in prompts:
    completion = generation_func(prompt)
    functions = extract_function_calls(completion)

    if functions:
        print(functions)
    else:
        print(completion.strip())
    print("="*100)



{"name": "Joke", "arguments": {"setup": "Why don't scientists trust atoms?", "answer": "Because they make up everything."}}<|im_end|>
{"name": "SongRecommendation", "arguments": {"interest": "inspiration", "recommended_song": "Eye of the Tiger by Survivor"}}<|im_end|>
{"name": "BookRecommendation", "arguments": {"interest": "Crime Thriller", "recommended_book": "The Girl on the Train by Paula Hawkins"}}<|im_end|>
